In [1]:
pip install scikit-learn==1.2.0

Note: you may need to restart the kernel to use updated packages.


## import packages

In [7]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

## function to predict the hand gesture

In [8]:
def predict(input):
    model_dict = pickle.load(open('./model.p', 'rb'))
    model = model_dict['model']
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
    labels_dict = {0: 'L', 1: 'A', 2: 'B', 3: 'c', 4: 'Hello'}

#while True:
    data_aux = []
    frame = input
    #frame = cv2.imread(input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
            frame,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        

        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x)
            data_aux.append(y)
    
    if data_aux:
        prediction = model.predict([np.asarray(data_aux)])
        predicted_character = labels_dict[int(prediction[0])]
        return frame, predicted_character
        # Process the prediction as needed
    else:
        return frame, "No hand landmarks detected in the current frame."
         

## UI for image input

In [11]:
demo = gr.Interface(fn=predict,
             inputs=gr.Image(label="camera input", show_label=True, sources = "webcam", type = "filepath"), 
             outputs = ["image", "text"], title = " Sign translation App")
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## function to predict hand gesture in each frame of the video

In [9]:
def vid_inf(vid):
    cap = cv2.VideoCapture(vid)  # Start capturing video from the file
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame, character  = predict(frame)  # predict the hand gesture in the frame
            yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), character, None  # Yield the processed frame
        else:
            break
            
    cap.release()  # Release the video capture object
    cv2.destroyAllWindows()  # Close all OpenCV windows

In [45]:
pip install ffmpeg-python

  Obtaining dependency information for ffmpeg-python from https://files.pythonhosted.org/packages/d7/0c/56be52741f75bad4dc6555991fabd2e07b432d333da82c11ad701123888a/ffmpeg_python-0.2.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [5]:
import ffmpeg

In [14]:
## UI for video input

In [17]:
input_video = gr.Video(sources = ["webcam"], label="Input Video")
output_frame = gr.Image(label="Output Frames")
output_character = gr.Textbox(label = "sign")
# Create Gradio Interface for Video Inference
interface_video = gr.Interface(
    fn=vid_inf,
    inputs=[input_video],
    outputs=[output_frame, output_character],
    title="Video Inference")
interface_video.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
